In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import time
import os
import random

In [2]:
    def read_data( hf, target_device, house_list):
        """Load data of houses
        """

        if len(house_list) == 0:
            return [], []

        else:
            aggregates = []
            targets = []
            #timestamps=[]
        
            for house in house_list:

                aggregate = hf[house]['aggregate'][:]
                target = hf[house][target_device][:]
                #timestamp=hf[house]['unix'][:]
                
                aggregates.append(aggregate)
                targets.append(target)
                #timestamps.append(timestamp)
            aggregates = np.concatenate(aggregates, axis=0)
            targets = np.concatenate(targets, axis=0)
            #times=np.concatenate(timestamps,axis=0)
            
            return aggregates, targets

In [3]:
def load_activation(path,ls):
    step = 8640
    dw_activations = []
    #path = 'app_activations/dishwasher/'
    for house in ls:
        folder_path = path+'h'+str(house)+'/'
        file_list = os.listdir(folder_path)
        for numpy_file in file_list:
            file_path = os.path.join(folder_path, numpy_file)
            tem_value=np.load(file_path)
            if len(tem_value)<8640:
                dw_activations.append(np.load(file_path))
    return dw_activations

In [4]:
def syn_data(application,w_activations,k_activations,m_activations,house):
    target_device=application
    train_house_list=[house]
    hf = h5py.File('experiment_data/data_washingmachine_mixture.h5', 'r')
    (aggregation, train_y) = read_data(hf, target_device, train_house_list)
    
    order=[]
    for i in range(0,len(train_y),8640):
        order.append(i-1)
    order[0]=0
    print(len(order))
    for i in range (1,len(order)):
        wr=random.choice(w_activations)
        #dr=random.choice(d_activations)
        kr=random.choice(k_activations)
        mr=random.choice(m_activations)
        
        synthetic_index1=random.randint(order[i-1],order[i]-len(wr))
        synthetic_index2=random.randint(order[i-1],order[i]-len(kr))
        synthetic_index3=random.randint(order[i-1],order[i]-len(mr))
        for j in range(0,len(wr)):
            aggregation[synthetic_index1+j]=aggregation[synthetic_index1+j]+wr[j]
        for j in range(0,len(kr)):
            aggregation[synthetic_index2+j]=aggregation[synthetic_index2+j]+kr[j]
        for j in range(0,len(mr)):
            aggregation[synthetic_index3+j]=aggregation[synthetic_index3+j]+mr[j]
    df=pd.DataFrame({application:train_y,'Aggregate':aggregation})
    path="data/"+house+".csv"
    df.to_csv(path, index=False, header=True)

    


In [5]:
# lw=[1,2,3,5,6,7,8,9,10,11]
# w_activations=load_activation('app_activations/washingmachine/',lw)

lm=[2,3,4,5,6,8,9]
m_activations=load_activation('app_activations/microwave/',lm)

lk=[2,3,4,5,6,7,8]
k_activations=load_activation('app_activations/kettle/',lk)

ld=[1,2,3,5,6,7,9]  
w_activations=load_activation('app_activations/dishwasher/',ld)

# for i in ["house1","house2","house3","house5","house6","house7","house9"]:
#     syn_data("dishwasher",w_activations,k_activations,m_activations,i)

# for i in ["house2","house3","house4","house5","house6","house8","house9"]:
#     syn_data("microwave",w_activations,k_activations,m_activations,i)

for i in ["house1","house2","house3","house5","house6","house7","house8","house9","house10","house11"]:
    syn_data("washingmachine",w_activations,k_activations,m_activations,i)

# for i in ["house2","house3","house4","house5","house6","house7","house8"]:
#     syn_data("kettle",w_activations,k_activations,m_activations,i)

639
618
615
649
578
614
556
569
587
393


In [6]:
def produce_data_vt(application,house):
    target_device=application
    train_house_list=[house]
    hf = h5py.File('data.h5', 'r')
    (train_x, train_y) = read_data(hf, target_device, train_house_list)
    df=pd.DataFrame({application:train_y,'Aggregate':train_x})
    path="data/"+house+".csv"
    df.to_csv(path, index=False, header=True)
    

In [7]:
# for i in ["house10","house11","house13","house15","house16","house18","house20"]:
#     produce_data_vt('dishwasher',i)

In [8]:
# for i in ["house10","house11","house12","house15","house17","house18","house19","house20"]:
#     produce_data_vt('microwave',i)

In [9]:
for i in ["house13","house15","house16","house17","house18","house19","house20"]:
    produce_data_vt('washingmachine',i)

In [10]:
# for i in ["house9","house11","house12","house13","house17","house19","house20"]:
#     produce_data_vt('kettle',i)